In [171]:
from pathlib import Path
desktop_path = Path.home() / "Desktop"
import sys # https://stackoverflow.com/questions/4383571/importing-files-from-different-folder
sys.path.insert(1, rf'{desktop_path}\RA\penicilin\Libaries')

import reload_lib as rl

import main as m
from results_figures import *

# from main import main as m
rl.reload_lib(m)

In [172]:
import pandas as pd
import numpy as np
from pathlib import Path

In [173]:
rl.reload_lib(m)
init = m.main()

In [174]:
lci_table_path = init.lci_table_path
lci_table_path

WindowsPath('c:/Users/ruw/Desktop/RA/penicilin/data/LCI_tables.xlsx')

In [175]:
init.results_path

WindowsPath('C:/Users/ruw/Desktop/RA/penicillin_results')

In [176]:
init.create_LCI_tables()

In [5]:
sheets_to_import = init.extract_excel_sheets()
data = pd.read_excel(init.system_path, sheet_name=sheets_to_import[0])
data

,Database,penicillin_cut_off_IN,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Activity,magnesium stearate,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,production amount,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,reference product,magnesium stearate,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,location,RoW,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
470,"Penicillin V, defined system",1,"Penicillin V, defined system",DK,unit,NaN,production,penicillin_cut_off_IN,NaN
471,packaging of a medicine strip,0.033333,packaged medicine strip,IN,unit,NaN,technosphere,penicillin_cut_off_IN,NaN
472,production of a medicin cup,1,medicin cup,RER,unit,NaN,technosphere,penicillin_cut_off_IN,NaN
473,"treatment of hazardous waste, hazardous waste ...",-0.002497,"hazardous waste, for incineration",CH,kilogram,NaN,technosphere,ev391cutoff,NaN


In [6]:
dcols = data.columns
db = None

# Improved logic: avoid nested loops, use vectorized search for columns first, then rows if needed
if db is None:
    # Search for column names containing both keywords
    for icol, col in enumerate(dcols):
        if "database" in str(col).lower():
            # Check if next column exists and matches
            if icol + 1 < len(dcols) and "penicillin_cut_off" in str(dcols[icol + 1]).lower():
                db = dcols[icol + 1]
                break
    # If not found in columns, search in rows
    if db is None:
        for irow, row in data.iterrows():
            for icol, col in enumerate(dcols):
                cell_val = str(row[col]).lower() if pd.notnull(row[col]) else ""
                if "database" in cell_val:
                    # Check if next column exists and matches
                    if icol + 1 < len(dcols):
                        next_cell_val = str(row[dcols[icol + 1]]).lower() if pd.notnull(row[dcols[icol + 1]]) else ""
                        if "penicillin_cut_off" in next_cell_val:
                            db = dcols[icol + 1]
                            break
            if db is not None:
                break
    
db

'penicillin_cut_off_IN'

In [119]:
with pd.ExcelFile(lci_table_path) as excel_file:
    # Get the sheet names
    sheet_names = excel_file.sheet_names
data = {}
for sheet in sheet_names:
    data[sheet] = pd.read_excel(init.lci_table_path, sheet_name=sheet)





In [ ]:
def bolding_input_biosphere_text(df_lci):
    for col in df_lci.columns:
        for idx, row in df_lci.iterrows():
            if "Input" in str(row[col]) or "Biosphere flow" in str(row[col]) and "bf" not in row[col]:
                df_lci.at[idx, col] = "\\bf{" + row[col] + "}"
            if "%" in str(row[col]):
                print(df_lci.at[idx, col])
                df_lci.at[idx, col] =  str(row[col]).replace("%","\\%")
                print(df_lci.at[idx, col])
            

In [88]:
init.path_github

WindowsPath('c:/Users/ruw/Desktop/RA/penicilin')

In [125]:
dfd = data["manufacturing of raw penicill G"]
dfd

,Reference Flow,Amount,Unit,Provider
0,raw penicillium G,1.000000,kilogram,manufacturing of raw penicillium G
1,Input,NaN,NaN,NaN
2,"acetone, liquid",0.220000,kilogram,"market for acetone, liquid | RoW"
3,ammonium sulfate,0.320000,kilogram,market for ammonium sulfate | RoW
4,butyl acetate,0.180000,kilogram,market for butyl acetate | RoW
5,"electricity, low voltage",21.977778,kilowatt hour,"market group for electricity, low voltage | IN"
6,glucose,5.180000,kilogram,glucose production | RoW
7,"heat, from steam, in chemical industry",6.715500,megajoule,"market for heat, from steam, in chemical indus..."
8,"oxygen, liquid",4.020000,kilogram,"market for oxygen, liquid | RoW"
9,pharmamedia,1.300000,kilogram,production of pharmamedia | IN


In [ ]:
def unit_conversion_lci_table(unit, unit_change):
    if "kilogram" in unit:
        unit = "kg"
    if "kilowatt hour" in unit:
        unit = "kWh"
    if "megajoule" in unit:
        unit = "MJ"
    if unit_change:
        if "k" in unit:
            unit = unit[1:]
        elif "MJ" in unit:
            unit = unit.replace("M","k")

    return unit

In [144]:
import numpy as np

In [ ]:
with pd.ExcelFile(lci_table_path) as excel_file:
    # Get the sheet names
    sheet_names = excel_file.sheet_names
data = {}
for sheet in sheet_names:
    data[sheet] = pd.read_excel(init.lci_table_path, sheet_name=sheet)

dfd = data["Penicillin G, defined system"]
for idx, row in dfd.iterrows():
    unit_change = False
    unit  = str(row["Unit"])
    amount =  row["Amount"]
    rf = row["Reference Flow"]
    if abs(amount) <= 0.1 and isinstance(amount, float) and "unit" not in unit:
        new_amount = amount * 1000
        dfd.at[idx, "Amount"] = round(new_amount,2)
        unit_change = True
        dfd.at[idx, "Unit"] = unit_conversion_lci_table(unit, unit_change)
    elif "Input" in rf or "Biosphere" in rf:
        continue
    else:
        dfd.at[idx, "Amount"] = round(amount,2)
        dfd.at[idx, "Unit"] = unit_conversion_lci_table(unit, unit_change)



,Reference Flow,Amount,Unit,Provider
0,"Penicillin G, defined system",1.00,unit,"Penicillin G, defined system"
1,Input,NaN,NaN,NaN
2,alchohol wipes,1.00,unit,production of alchohol wipes | DK
3,full box with glass vials,0.10,unit,packaging of glass vials with penicillin G | IN
4,pair of gloves,1.00,unit,production of a pair of gloves | MY
5,"hazardous waste, for incineration",-82.80,g,"treatment of hazardous waste, hazardous waste ..."
6,IV bag,1.00,unit,production of IV bag | RER
7,medical connector,1.00,unit,medical connector | GLO
8,"sodium chlorate, powder",0.90,g,"market for sodium chlorate, powder | RER"
9,stopcock,1.00,unit,stopcock | GLO


In [ ]:
pd.DataFrame().to_latex(float_format=2)

'\\begin{tabular}{l}\n\\toprule\n\\midrule\n\\bottomrule\n\\end{tabular}\n'

In [ ]:
final_txt = ""
for sheet_name, lci_table in data.items():

    bolding_input_biosphere_text(lci_table)

    latex_str = lci_table.to_latex(
        escape=False,
        index=False,
        column_format='p{5.75cm}p{1.75cm}p{1.25cm}p{6.75cm}@{}',
        na_rep="",
        caption=f"Life Cycle Inveotry : {lci_table.iat[0,3]}",
        label=f"tab:lci:{lci_table.iat[0,0]}",
          
    )
    # Split into lines
    lines = latex_str.splitlines()

    # Find where the data rows start (after \midrule)
    midrule_index = next(i for i, line in enumerate(lines) if r'\midrule' in line)

    # Insert extra \midrule after the third data row
    insert_at_line_2 = midrule_index + 2  # 3 rows after \midrule
    lines.insert(insert_at_line_2, r'\midrule')

    # Insert extra \midrule after the third data row
    insert_at_line_4 = midrule_index + 4  # 3 rows after \midrule
    lines.insert(insert_at_line_4, r'\midrule')

    header_index = midrule_index - 1
    header_row = lines[header_index]

    # Bold each header cell
    bold_headers = [f"\\textbf{{{cell.strip()}}}" for cell in header_row.split('&')]
    lines[header_index] = ' & '.join(bold_headers) + r' \\'

    lines.append('\n')

    # Join back into a string
    custom_latex = '\n'.join(lines)
    final_txt += custom_latex

final_txt

text_file = open("Output.txt", "w")

text_file.write(final_txt)

text_file.close()

polystyrene foam slab production, 100% recycled | CH
polystyrene foam slab production, 100\% recycled | CH
sodium hydroxide, without water, in 50% solution state
sodium hydroxide, without water, in 50\% solution state
market for sodium hydroxide, without water, in 50% solution state | GLO
market for sodium hydroxide, without water, in 50\% solution state | GLO
sodium hydroxide, without water, in 50% solution state
sodium hydroxide, without water, in 50\% solution state
market for sodium hydroxide, without water, in 50% solution state | GLO
market for sodium hydroxide, without water, in 50\% solution state | GLO
ethanol, without water, in 99.7% solution state, from ethylene
ethanol, without water, in 99.7\% solution state, from ethylene
market for ethanol, without water, in 99.7% solution state, from ethylene | RER
market for ethanol, without water, in 99.7\% solution state, from ethylene | RER
polystyrene foam slab production, 100% recycled | CH
polystyrene foam slab production, 100\% 

In [31]:
latex_str = data["Penicillin V, defined system"].to_latex(
    escape=False,
    index=False,
    column_format='p{5.75cm}p{1.75cm}p{1.25cm}p{6.75cm}@{}',
    na_rep="",
    bold_rows=True,
    caption=f"Life Cycle Inveotry : {data['Penicillin V, defined system'].iat[0,3]}",
    label=f"tab:lci:{data['Penicillin V, defined system'].iat[0,0]}",    
)
# Split into lines
lines = latex_str.splitlines()

# Find where the data rows start (after \midrule)
midrule_index = next(i for i, line in enumerate(lines) if r'\midrule' in line)

# Insert extra \midrule after the third data row
insert_at_line_2 = midrule_index + 2  # 3 rows after \midrule
lines.insert(insert_at_line_2, r'\midrule')

# Insert extra \midrule after the third data row
insert_at_line_4 = midrule_index + 4  # 3 rows after \midrule
lines.insert(insert_at_line_4, r'\midrule')

header_index = midrule_index - 1
header_row = lines[header_index]

# Bold each header cell
bold_headers = [f"\\textbf{{{cell.strip()}}}" for cell in header_row.split('&')]
lines[header_index] = ' & '.join(bold_headers) + r' \\'

lines.append('\n')

# Join back into a string
custom_latex2 = '\n'.join(lines)
#     final_txt += custom_latex

custom_latex2

'\\begin{table}\n\\caption{Life Cycle Inveotry : Penicillin V, defined system}\n\\label{tab:lci:Penicillin V, defined system}\n\\begin{tabular}{p{5.75cm}p{1.75cm}p{1.25cm}p{6.75cm}@{}}\n\\toprule\n\\textbf{Reference Flow} & \\textbf{Amount} & \\textbf{Unit} & \\textbf{Provider \\\\} \\\\\n\\midrule\nPenicillin V, defined system & 1.000000 & unit & Penicillin V, defined system \\\\\n\\midrule\nInput &  &  &  \\\\\n\\midrule\npackaged medicine strip & 0.033333 & unit & packaging of a medicine strip | IN \\\\\nmedicin cup & 1.000000 & unit & production of a medicin cup | RER \\\\\nhazardous waste, for incineration & -0.002497 & kilogram & treatment of hazardous waste, hazardous waste incineration | CH \\\\\nwaste packaging paper & -0.002632 & kilogram & waste packaging paper, Recycled Content cut-off | GLO \\\\\n\\bottomrule\n\\end{tabular}\n\\end{table}\n\n'

In [32]:
custom_latex = custom_latex1 + custom_latex2
custom_latex


'\\begin{table}\n\\caption{Life Cycle Inveotry : Penicillin G, defined system}\n\\label{tab:lci:Penicillin G, defined system}\n\\begin{tabular}{p{5.75cm}p{1.75cm}p{1.25cm}p{6.75cm}@{}}\n\\toprule\n\\textbf{Reference Flow} & \\textbf{Amount} & \\textbf{Unit} & \\textbf{Provider \\\\} \\\\\n\\midrule\nPenicillin G, defined system & 1.000000 & unit & Penicillin G, defined system \\\\\n\\midrule\nInput &  &  &  \\\\\n\\midrule\nalchohol wipes & 1.000000 & unit & production of alchohol wipes | DK \\\\\nfull box with glass vials & 0.100000 & unit & packaging of glass vials with penicillin G | IN \\\\\npair of gloves & 1.000000 & unit & production of a pair of gloves | MY \\\\\nhazardous waste, for incineration & -0.082795 & kilogram & treatment of hazardous waste, hazardous waste incineration, with energy recovery | CH \\\\\nIV bag & 1.000000 & unit & production of IV bag | RER \\\\\nmedical connector & 1.000000 & unit & medical connector | GLO \\\\\nsodium chlorate, powder & 0.000900 & kilo

In [111]:
custom_latex

''